In [1]:
!mkdir -p /data/sets/nuscenes  # Make the directory to store the nuScenes dataset in.

!wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.
!wget https://www.nuscenes.org/data/nuScenes-lidarseg-mini-v1.0.tar.bz2  # Download the nuScenes-lidarseg mini split.

!tar -xf v1.0-mini.tgz -C /data/sets/nuscenes  # Uncompress the nuScenes mini split.
!tar -xf nuScenes-lidarseg-mini-v1.0.tar.bz2 -C /data/sets/nuscenes   # Uncompress the nuScenes-lidarseg mini split.
!pip install nuscenes-devkit &> /dev/null

--2023-10-20 18:23:54--  https://www.nuscenes.org/data/v1.0-mini.tgz
Resolving www.nuscenes.org (www.nuscenes.org)... 13.227.219.76, 13.227.219.6, 13.227.219.18, ...
Connecting to www.nuscenes.org (www.nuscenes.org)|13.227.219.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4167696325 (3.9G) [application/x-tar]
Saving to: ‘v1.0-mini.tgz’

v1.0-mini.tgz       100%[===================>]   3.88G  86.0MB/s    in 46s     

2023-10-20 18:24:40 (86.9 MB/s) - ‘v1.0-mini.tgz’ saved [4167696325/4167696325]

--2023-10-20 18:24:40--  https://www.nuscenes.org/data/nuScenes-lidarseg-mini-v1.0.tar.bz2
Resolving www.nuscenes.org (www.nuscenes.org)... 13.227.219.6, 13.227.219.76, 13.227.219.45, ...
Connecting to www.nuscenes.org (www.nuscenes.org)|13.227.219.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819477 (1.7M) [application/x-tar]
Saving to: ‘nuScenes-lidarseg-mini-v1.0.tar.bz2’

nuScenes-lidarseg-m 100%[===================>]   1.73M  --

In [2]:
from nuscenes import NuScenes
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper
import matplotlib.pyplot as plt
import numpy as np
from nuscenes.prediction import PredictHelper
DATAROOT = '/data/sets/nuscenes'
ns = NuScenes('v1.0-mini', dataroot=DATAROOT , verbose=True)
helper = PredictHelper(ns)
import os
os.chdir('..')
!pwd

Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.841 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
/


In [ ]:
ped_cat = ['human.pedestrian.adult','human.pedestrian.child','human.pedestrian.construction_worker',
           'human.pedestrian.personal_mobility',
           'human.pedestrian.police_officer']
def calculate_ped_trajectory(ped_cat):
    count = 0
    dict_of_trajectory = {}
    selected_pedestrain = []
    for attr in ns.sample_annotation:

      if attr['category_name'] in ped_cat:
        if len(attr['prev']) < 2 :
          token          = attr['token']
          sample_token   = attr['sample_token']
          instance_token = attr['instance_token']
          future_xy_local = helper.get_future_for_agent(instance_token, sample_token, seconds=20, in_agent_frame=False , just_xy=False)
          dict_of_trajectory[attr['instance_token'] +'_'+ attr['sample_token']] = future_xy_local

    list_of_trajectory = list(dict_of_trajectory.values())
    return list_of_trajectory ,dict_of_trajectory
#list of all trajectory in the mini_nuscenes dataset
list_of_trajectory  , dict_of_trajectory = calculate_ped_trajectory(ped_cat)
list_of_trajectory


In [4]:
def grouped_scene_seq_func(list_of_trajectory):
    scene_seq_list = []
    for i in range(len(list_of_trajectory)):
        sub_list = []
        if len(list_of_trajectory[i]) == 0:
          continue
        sample_token = list_of_trajectory[i][0]['sample_token']
        #find scene token for trajectories
        scene_token = ns.get('sample' , sample_token)['scene_token']
        sub_list.append(scene_token)
        sub_list.append(list_of_trajectory[i])
        #list that first item is scene token and second item is trajectory in that scene
        scene_seq_list.append(sub_list)
    #groupby according to scene_token((all trajectory in the scene now are in same list))
    grouped_scene_seq = {}
    for sublist in scene_seq_list:
        key = sublist[0]
        if key in grouped_scene_seq.keys():
            grouped_scene_seq[key].append(sublist[1])
        else:
            grouped_scene_seq[key] = [sublist[1]]
    return grouped_scene_seq
#scene_seq_list is a dictionary that key is scene token and values are all pedestrain trajectory in this key
grouped_scene_seq = grouped_scene_seq_func(list_of_trajectory)

In [5]:
scene_agg_list = []
for scene_token in grouped_scene_seq.keys():
      sequences = grouped_scene_seq[scene_token]
      agg_seq = []
      for sequence in sequences:
          instance_token , sample_token =   sequence[0]['instance_token'] , sequence[0]['sample_token']
          past = helper.get_past_for_agent(instance_token ,sample_token , seconds=20 ,in_agent_frame=False )
          future = helper.get_future_for_agent(instance_token  , sample_token , seconds=20 ,in_agent_frame=False )
          if len(future) ==0:
            continue
          seq = np.append(past ,future , axis=0)
          agg_seq.append(seq)
      scene_agg_list.append(agg_seq)
#creat file for each scene(each file contain all pedestrain and trajectory according to eth and ucy format)
      scene  = scene_token
      for temp in scene_agg_list:
          file_path = f'/content/{scene} +.txt'
          ped_id = 0
          with open (file_path , 'a') as f:
            for seq in temp:
                ped_id+=1
                for frame_number , xy in enumerate(seq):
                    f.write(f"{frame_number}  {ped_id}  {xy[0]}  {xy[1]}\n")
          with open(f'/content/{scene} +.txt', 'r') as file:
            lines = file.readlines()

          rows = [line.strip().split() for line in lines]
          sorted_rows = sorted(rows, key=lambda x: int(x[0]))
          sorted_lines = [' '.join(row) + '\n' for row in sorted_rows]

          # Save the sorted data to a new file
          with open(f'/content/scene_id:{scene}+.txt', 'w') as output_file:
              output_file.writelines(sorted_lines)
              os.remove(f'/content/{scene} +.txt')